In [1]:
import numpy as np
from sim_handling import Simulation,loadSim, continue_from_file #uses ds7, both PDES leads to stable solution
from diffusionstuff7 import f0d,f1d,f2d, getsigmastep_2d

# for plotting
from matplotlib import pyplot as plt
from matplotlib import rcParams
from matplotlib.animation import PillowWriter
from  matplotlib.ticker import FormatStrFormatter

#for handling large simulations writing/reading from disk
import os
#import csv
import psutil

C:\Users\Max-PC\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\llvmlite\llvmpy\__init__.py:3: UserWarning: The module `llvmlite.llvmpy` is deprecated and will be removed in the future.
  warnings.warn(
C:\Users\Max-PC\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\llvmlite\llvmpy\core.py:8: UserWarning: The module `llvmlite.llvmpy.core` is deprecated and will be removed in the future. Equivalent functionality is provided by `llvmlite.ir`.
  warnings.warn(
C:\Users\Max-PC\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\llvmlite\llvmpy\passes.py:17: UserWarning: The module `llvmlite.llvmpy.passes` is deprecated and will be removed in the future. If you are using this code, it should be inlined into your own project.
  warnings.warn(


In [ ]:
%matplotlib widget

#set figsize to be big (12,12) inches
#rcParams['figure.figsize'] = 10,10

assymettric 2d growth

In [ ]:
#load a simulation

#load the ablation with initial noise run
#test_2d_asym = loadSim('f2d_simulation_asymmetric250_200_500layers_center_reduction0.1_D_2_RK45_sigmamax_-0.01noisyinit_True.pkl')

#load the growth with initial noise run
#test_2d_asym = loadSim('f2d_simulation_asymmetric250_200_500layers_center_reduction0.1_D_2_RK45_sigmamax_0.2noisyinit_True.pkl')


In [ ]:
#set up and run and plot a new simulation
test_2d_asym = Simulation(f2d, (500,500), method='RK45')

test_2d_asym.discretization_halt = False

#try a noisy simulation
test_2d_asym.noisy_init = False

#test_2d_asym.float_params['DoverdeltaX2'] *= 2 
test_2d_asym.layermax = 700
#test_2d_asym.sigmastepmax= -0.02#ablation
test_2d_asym.sigmastepmax= 0.20 #growth
#test_2d_asym.center_reduction = 0.25
#test_2d_asym.deltaT *= 10 #increase time step to decrease number of steps saved

#_plot = test_2d_asym.plot(tot=True,completion=1)
#anim = test_2d_asym.animate(ice=True,surface=True, ret=True, focus_on_growth=True)

In [ ]:
my_id=['prog_report_large_2d'+str(test_2d_asym.shape[0])+'_'+str(test_2d_asym.shape[1]),str(test_2d_asym.layermax)+'layers',
                                'center_reduction'+str(test_2d_asym.center_reduction),  'D_2', test_2d_asym.method, 'sigmamax_'+str(test_2d_asym.sigmastepmax) +'noisyinit_'+str(test_2d_asym.noisy_init)] #'Cy_0_Dy0',
my_filename = ''.join([i+'_' for i in my_id])

In [ ]:
#calcs to graph asymmetric sigmastep2d
xs,ys = test_2d_asym.x, test_2d_asym.y #linspace
sigma = getsigmastep_2d(xs,ys, test_2d_asym.center_reduction, test_2d_asym.sigmastepmax)
ygrid,xgrid = np.meshgrid(ys,xs) #2d mesh for plotting

In [ ]:
#countour plot of sigmastep 2d
print(xs.shape,ys.shape,sigma.shape)
fig,ax = plt.subplots()
CS = ax.contourf(xgrid,ygrid,sigma*100,np.arange(np.min(sigma*100)-0.01,np.max(sigma*100)+0.01,0.01))
#CS = ax.contourf(xgrid,ygrid,xgrid) #graph x
#CS = ax.contourf(xgrid,ygrid,ygrid) #graph y
ax.set_xlabel(r'$x (\mu m)$', fontsize=20)
ax.set_ylabel(r'$y (\mu m)$', fontsize=20)
plt.grid(True)
plt.gca().set_aspect('equal')
fig.colorbar(CS)
plt.title('%supersaturation (getsigmastep_2d)')
print(xgrid.shape,ygrid.shape)

In [ ]:
#animate the simulation (runs if needed)
#test_2d_asym.run(halve_time_res=True) #only saves half the time steps since it is a huge simulation

anim = test_2d_asym.animate(ret=True, focus_on_growth=True, proportionalSpeed=False)

In [ ]:
#start a second simulation starting with the ending surface of the first 
# as the initial surface

#create 1d sim and graph number of steps over time
cycle_sim =  Simulation(f2d, (test_2d_asym.shape[0],test_2d_asym.shape[1]), method=test_2d_asym.method, rtol=test_2d_asym.rtol)

#keep general params same
cycle_sim.layermax = test_2d_asym.layermax * 1.5 #make sure it reaches steady state
cycle_sim.float_params['DoverdeltaX2'] = test_2d_asym.float_params['DoverdeltaX2'] 
cycle_sim.sigma0 = test_2d_asym.sigma0
cycle_sim.center_reduction = test_2d_asym.center_reduction
cycle_sim.deltaT = test_2d_asym.deltaT #increase time step to decrease number of steps saved?
cycle_sim.noisy_init = test_2d_asym.noisy_init

#set up initial surface
cycle_sim.nonstd_init = True
ntots = test_2d_asym.getNtot()
cycle_sim.startingNtot = ntots[len(ntots)//2] #start with the middle of the last simulation, before discretzation error

#set to ablation run
cycle_sim.sigmastepmax = -0.01


In [ ]:
#animate ablation of growth ending surface
anim = cycle_sim.animate(ret=True, focus_on_growth=True, proportionalSpeed=False)

In [ ]:
# #profile the running and processing of the simulation
# import cProfile
# import pstats

# def my_function():
#     #graph number of steps over time
#     #handle large sims
#     filename = 'long_2D_sim_500_500_pts.csv'
#     Ntot = handle_mem_intense_sim(test_2d_asym, filename)

#     layers_grown_at_each_timestep = []
#     max_less_min = []
#     if Ntot is None:
#         timesteps = 0
#         #read from csv file
#         with open(filename, 'r') as f:
#             Ntot = np.load(f, mmap_mode='r')[:,1,:,:]
#             for row in Ntot:
#                 timesteps += 1 #count number of timesteps
#                 max_less_min.append([np.max(n)-np.min(n) for n in row])
#                 layers_grown_at_each_timestep.append([np.max(n) for n in row])
#                 del row #remove row from memory to prevent memory error

#         max_less_min = np.array(max_less_min)#.reshape(-1, cycle_sim.shape[0], cycle_sim.shape[1])
#     else:
#         for row in Ntot:
#             max_less_min.append(np.max(n)-np.min(n) for n in row)

#         max_less_min = np.array(max_less_min)#.reshape(-1, cycle_sim.shape[0], cycle_sim.shape[1])
#         timesteps = len(Ntot)

#     # maxes = np.array([np.max(n) for n in Ntot[:]])
#     # mins = np.array([np.min(n) for n in Ntot[:]])
#     #print(maxes,mins)
#     return max_less_min,timesteps,Ntot,layers_grown_at_each_timestep,filename

# # Create a Profile object
# pr = cProfile.Profile()

# # Run the function and create the profile data
# max_less_min,timesteps,Ntot,layers_grown_at_each_timestep,filename = pr.runcall(my_function)

# # Create a Stats object
# stats = pstats.Stats(pr)

# # Print out the statistics
# stats.strip_dirs().sort_stats("time").print_stats()


In [ ]:
#graph number of steps over time
#handle large sims
filename = 'long_2D_sim_500_500_pts.npy'
test_2d_asym.filename = filename
run_filename_output = test_2d_asym.run()
if run_filename_output is not None:
    next_sim = continue_from_file(test_2d_asym, filename)
    inner_run_filename_output = next_sim.run()
    while inner_run_filename_output is not None:
        next_sim = continue_from_file(next_sim, filename)
        inner_run_filename_output = next_sim.run()
        
if run_filename_output is None:
    Ntot = test_2d_asym.getNtot()
else:
    Ntot = np.load(filename, mmap_mode='r')[:,1,:,:]

#Ntot = handle_mem_intense_sim(test_2d_asym, filename)

#layers_grown_at_each_timestep = []
max_less_min = []
if Ntot is None:
    timesteps = 0
    #read from csv file
    with open(filename, 'r') as f:
        Ntot = np.load(f, mmap_mode='r')[:,1,:,:]
        for row in Ntot:
            timesteps += 1 #count number of timesteps
            #max_less_min.append([np.max(n)-np.min(n) for n in row])
            max_less_min.append(np.max(row)-np.min(row))

            #layers_grown_at_each_timestep.append([np.max(n) for n in row])
            del row #remove row from memory to prevent memory error

    max_less_min = np.array(max_less_min)#.reshape(-1, cycle_sim.shape[0], cycle_sim.shape[1])
else:
    for row in Ntot:
        max_less_min.append(np.max(row)-np.min(row))

    max_less_min = np.array(max_less_min)#.reshape(-1, cycle_sim.shape[0], cycle_sim.shape[1])
    timesteps = len(Ntot)

# maxes = np.array([np.max(n) for n in Ntot[:]])
# mins = np.array([np.min(n) for n in Ntot[:]])
#print(maxes,mins)

In [ ]:
print(psutil.virtual_memory())
print(psutil.swap_memory())

In [ ]:
#optionally save Ntot so no more re runs
save_Ntot = False
if save_Ntot: np.save('Ntot.npy', Ntot)

In [ ]:
#print(layers_grown_at_each_timestep) 

In [ ]:
nogen_mlm = np.array([list(x) for x in max_less_min])
np.array([list(x) for x in max_less_min])[:,0]

In [ ]:
print(nogen_mlm)

In [ ]:
#plot the number of steps over time
plt.figure()
ax = plt.subplot(111)

time_axis = np.asarray(np.arange(timesteps))

ax.set_ylabel('Number of steps on surface')

if test_2d_asym.sigmastepmax < 0: #ablation case
    ax.set_xlabel('Total number of layers ablated')
    ax.plot( Ntot[::-1,0,0]+test_2d_asym.layermax, max_less_min)#maxes-mins) #flipping to show ablation progress from left to right, adding layermax to show # layers ablated instead of negative # layers grown
else: #growth case   
    ax.set_xlabel('Total number of layers grown')
    #ax.plot(time_axis/max(time_axis)*test_2d_asym.layermax, np.asarray(nogen_mlm)[:,0])#maxes-mins)
    #ax.plot(time_axis/max(time_axis)*test_2d_asym.layermax, np.array([list(x) for x in max_less_min])[:,0])
    ax.plot(time_axis/max(time_axis)*test_2d_asym.layermax, max_less_min)
    #ax.plot( Ntot[:,0,0], maxes-mins)

plt.grid()

In [ ]:
#plot subplots of the 2d surface at different times

# from mpl_toolkits.mplot3d import Axes3D
    
#fig, axs = plt.subplots(2, 4)#, projection='3d')

fig = plt.figure()
axs = np.empty((2,4), dtype=object)

#adjust 3d view if desired
# for ax in axs.flatten():
    # ax.view_init(elev=30, azim=45)
    # ax.set_xlim(left=-50, right=50)
    # ax.set_ylim(bottom=-50, top=50)



for i, cmpltion in enumerate([0,0.01,0.03,0.05,0.1,0.3,0.5,0.8]):
    ax = fig.add_subplot(2, 4, i+1, projection='3d')
    axs[i//4, i%4] = ax

    if os.path.exists(filename):
        ntot = np.load(filename, mmap_mode='r')[:,1,:,:]
    else:
        ntot = Ntot

    zs = ntot[int(cmpltion*ntot.shape[0])]

    x,y = zs.shape[0],zs.shape[1]
    xs = np.linspace(x//-20,x//20,x)
    ys = np.linspace(y//-20,y//20,y)

    ys,xs = np.meshgrid(ys,xs)
    ax = axs[i//4, i%4]
    #.add_subplot(111, projection='3d')
    
    ax.set_xlabel(r'x ($\mu m$)')
    ax.set_ylabel('Layers of ice')
    ax.set_ylim(bottom=np.min(xs)-.5, top=np.max(xs)+.5)
    ax.yaxis.set_major_formatter(FormatStrFormatter('%.1f'))
    ax.set_title('Completion = '+str(cmpltion*100)+'%')
    ax.plot_surface(xs, ys, zs)


plt.show()


In [ ]:
#save the animation as a gif, trying this since Simulation.save_animation is not working
#anim.save(my_filename+'.gif',writer=PillowWriter(fps=60))



In [ ]:
#save animation with built in Simulation method
test_2d_asym.save_animation(my_filename, 'gif')

In [ ]:
#plot at specific timesteps
#cmpltion = 0.005 
#cmpltion = 0.1
#sim = test_2d_asym
sim = cycle_sim

for cmpltion in [0.9]:#[0.33,0.50,0.55,0.66,0.8,0.9]:  #[0,0.001,0.005]: # making plots of initial noise dissipating
    plt.close()
    sim.plot(completion=cmpltion, ice=True, figurename=my_filename+'_completion'+str(cmpltion))
    #plt.savefig(my_filename+'_completion'+str(cmpltion)+'.png')

In [ ]:
plt.savefig(''.join([i+'_' for i in my_id])+'.png')

In [ ]:
#save the simulation to a pickle file for later use (without re-running the simulation)
test_2d_asym.save(id=my_id)

#manually save it and add compression since the 1000 layter simulation is too big
# import pickle
# import bz2
# id=['asymmetric'+str(test_2d_asym.shape[0])+'_'+str(test_2d_asym.shape[1]),str(test_2d_asym.layermax)+'layers']
# id = ''.join('_'+i for i in id)
# filename = test_2d_asym.model.__name__+'_simulation'+id+'.pkl'
# with open(filename, 'wb') as f:
#     print("saving to", f)
#     pickle.dump(test_2d_asym, bz2.BZ2File(f, 'w'))

1D model for comparison (uses same parameters as above 2d model), graphing if it also reaching steady state

In [ ]:
#create 1d sim and graph number of steps over time
#ss_1d = Simulation(f1d, (test_2d_asym.shape[0],), method=test_2d_asym.method, rtol=test_2d_asym.rtol)
ss_1d = Simulation(f1d, (500,), method=test_2d_asym.method, rtol=test_2d_asym.rtol)

ss_1d.float_params['DoverdeltaX2'] = test_2d_asym.float_params['DoverdeltaX2'] 
ss_1d.layermax = test_2d_asym.layermax * 1.5 #make sure it reaches steady state
ss_1d.sigmastepmax = test_2d_asym.sigmastepmax 
ss_1d.sigma0 = test_2d_asym.sigma0
ss_1d.center_reduction = test_2d_asym.center_reduction
ss_1d.deltaT = test_2d_asym.deltaT #increase time step to decrease number of steps saved?
ss_1d.noisy_init = test_2d_asym.noisy_init


In [ ]:
#set the id 
my_id=['nx_'+str(ss_1d.shape[0]),str(ss_1d.layermax)+'layers',
                                'center_reduction'+str(ss_1d.center_reduction),  'D_2', ss_1d.method, 'sigmamax_'+str(ss_1d.sigmastepmax) +'noisyinit_'+str(ss_1d.noisy_init)]
my_filename = ''.join([i+'_' for i in my_id])

In [ ]:
# plot number of steps over time
fignum = 0
plt.close(fignum)

Ntot = ss_1d.getNtot()

maxes = np.array([np.max(n) for n in Ntot[:]])
mins = np.array([np.min(n) for n in Ntot[:]])

#print(maxes,mins)
plt.figure(fignum)
ax = plt.subplot(111)

#ax.plot(test_f2d_sim.results()['t'],maxes-mins)
ax.plot(Ntot[:,0],maxes-mins)
ax.set_ylabel('Number of steps on surface')
ax.set_xlabel('Time')
plt.grid()
plt.show()

In [ ]:
plt.close('1d surface over time 3d surface')
plt.figure('1d surface over time 3d surface')
#plot surface over time as 3d surface
ax = plt.axes(projection='3d')

#TODO:

zs = ss_1d.getNtot()
start,stop = 0, zs.shape[0]
#crop time selection
start,stop = stop*5//6, stop
zs = zs[start:stop]

t,x = [*zs.shape]
print(x,t)
xs,ys = np.meshgrid(np.linspace(0,x,x),np.linspace(start,stop,stop-start))

#print shapes of inputs
print(xs.shape,ys.shape,zs.shape)

#contour levels
levels = np.linspace(740,880,10)

#plot contours of the surface
ax.contour(X=xs, Y=ys, Z=zs,  label='ice+QLL', cmap='viridis_r')

#plot the surface with reversed viridis
ax.plot_surface(X=xs, Y=ys, Z=zs,  label='ice+QLL', cmap='viridis_r')

ax.set_xlabel('nx')
ax.set_ylabel('Time steps')
#ax.set_xlabel(r'x ($\mu m$)')
#ax.set_ylabel('Layers of ice')
plt.show()

In [ ]:
#plot contours over time on one surface
ax = plt.axes(projection='3d')

zs = ss_1d.getNtot()
start,stop = 0, zs.shape[0]
#crop time selection
#start,stop = stop*5//6, stop
zs = zs[start:stop]

t,x = [*zs.shape]
print(x,t)
xs,ys = np.meshgrid(np.linspace(0,x,x),np.linspace(start,stop,stop-start))

#print shapes of inputs
print(xs.shape,ys.shape,zs.shape)

# #contour levels
# levels = np.linspace(740,880,10)

# #plot the surface with reversed viridis
# ax.contour(X=xs, Y=ys, Z=zs,  label='ice+QLL', cmap='viridis_r')

# ax.set_xlabel('nx')
# ax.set_ylabel('Time steps')
# #ax.set_xlabel(r'x ($\mu m$)')
# #ax.set_ylabel('Layers of ice')
# plt.show()

ix = 100
plt.figure()

ixlist = [0, 10_000, 30_000, 40_000]#np.linspace(0,7400, 100)
offset = 1
for ix in ixlist:
    tempzs = zs[ix,:] - min(zs[ix,:]) + offset
    offset += 1
    plt.plot(xs[ix,:], tempzs,label = 'x = '+str(ix))

#plt.legend()

In [ ]:
#plot the surface over time on subplots

#run the simulation
ntot = ss_1d.getNtot()

fig, axs = plt.subplots(2, 4)
for i, cmpltion in enumerate([0,0.01,0.03,0.05,0.1,0.3,0.5,0.8]):

    xs = ntot[int(cmpltion*ntot.shape[0]),:]
    axs[i//4, i%4].set_xlabel(r'x ($\mu m$)')
    axs[i//4, i%4].set_ylabel('Layers of ice')
    axs[i//4, i%4].set_ylim(bottom=min(xs)-.5, top=max(xs)+.5)
    axs[i//4, i%4].yaxis.set_major_formatter(FormatStrFormatter('%.1f'))
    axs[i//4, i%4].set_title('Completion = '+str(cmpltion*100)+'%')
    axs[i//4, i%4].plot(xs, label='Completion = '+str(cmpltion*100)+'%')
plt.show()


In [ ]:
ss_1d.animate()

In [ ]:
plt.close()

In [ ]:
#test_2d_asym.save(id=['asymmetric'+str(test_2d_asym.shape[0])+'_'+str(test_2d_asym.shape[1]),str(test_2d_asym.layermax)+'layers'])
#symm = Simulation(f2d,(500,500),rtol=1e-12)

In [ ]:
#countour plot of sigmastep 2d
plt.figure()
ax = plt.axes(projection='3d')
plt.xlabel(r'x ($\mu m$)')
plt.ylabel(r'y ($\mu m$)')
#ax.set_zlabel('Layers of ice')
#levels = np.arange(19,21,0.1)
ax.contourf(xgrid,ygrid, sigma*100, extent=(0, 2, 0, 2), cmap='YlGnBu_r', vmin=0, vmax=200, zorder=1)#, levels=levels)

In [ ]:
#countour plot of sigmastep 2d
print(xs.shape,ys.shape,sigma.shape)
fig,ax = plt.subplots()
CS = ax.contourf(xs.T,ys.T,sigma*100)
ax.set_xlabel(r'$x (\mu m)$', fontsize=20)
ax.set_ylabel(r'$y (\mu m)$', fontsize=20)
plt.grid(True)
plt.gca().set_aspect('equal')
fig.colorbar(CS)
plt.title('%supersaturation (getsigmastep_2d)')

2D Growth simulation

In [ ]:
#create the 2d sim
test_f2d_sim = Simulation(f2d, (250,250), rtol=1e-12)
test_f2d_sim.layermax = 50#0
#test_f2d_sim.run()
#test_f2d_sim.save(_id=['test','50layer'])

In [ ]:
#animate the 2d sim
test_f2d_sim.animate(crossSection=True)

In [ ]:
#print variables in 1d simulation for comparison to literature
for k,v in test_f2d_sim._extra_vars.items():
    print(k,v)
print('center reduction %: ', test_f2d_sim.center_reduction)

In [ ]:
#print number of steps over time
Ntot = test_f2d_sim.getNtot()

maxes = np.array([np.max(n) for n in Ntot[:]])
mins = np.array([np.min(n) for n in Ntot[:]])

#print(maxes,mins)
plt.figure()
ax = plt.subplot(111)

#ax.plot(test_f2d_sim.results()['t'],maxes-mins)
ax.plot(Ntot[:,0,0],maxes-mins)
#ax.plot(test_f2d_sim.results()['t'],test_f2d_sim.results()['t'])
plt.grid()

In [ ]:
#animate test_f2d_sim

test_f2d_sim.animate(ice=True)

In [ ]:
#test_f2d_sim = loadSim('f2d_simulation_test_50layer.pkl') 

In [ ]:
#graph results normalized to minimum of ice surface
Fliq_ss, Ntot_steadystate = test_f2d_sim.normalize_results_to_min()
nice_ss = Ntot_steadystate-Fliq_ss
print(np.shape(Ntot_steadystate))
xs, ys = np.meshgrid(test_f2d_sim.x, test_f2d_sim.y)

fig = plt.figure(figsize=(12,12))
ax = plt.axes(projection = '3d')
#ax.plot_surface(test_f2d_sim._results['t'], Ntot_steadystate[:,])
ax.set_xlabel(r'$x (\mu m$)')#,fontsize=fontsize)
ax.set_ylabel(r'$y (\mu m$)')#,fontsize=fontsize)
ax.set_zlabel(r'$ice \ layers$')#,fontsize=fontsize)
#limits
#ax.set_zlim3d(-self.layermax, self.layermax)
#ax.set_ylim(0, max(self.y))
#ax.set_xlim(0, max(self.x))
def update_fig(num):
    ax.clear() # remove last iteration of plot 
    plot_func = ax.plot_surface
    # if crossSection:
    #     xmid = round(np.shape(Nice)[0]/2)
    #     if ice:
    #         plot_func(X=xs[xmid:], Y=ys[xmid:], Z=nice_ss[num][xmid:][:],cmap ='viridis')# cmap='viridis')#, vmin=0, vmax=200) #plot half of the surface of the ice
    #     if tot:
    #         plot_func(X=xs[xmid:], Y=ys[xmid:], Z=Ntot_steadystate[num][xmid:][:], cmap='cividis')#, vmin=0, vmax=200) #plot half the surface of the QLL
    # else:
    #     if ice:
    plot_func(X=xs, Y=ys, Z=nice_ss[num], cmap='viridis')#, vmin=0, vmax=200) #plot the surface of the ice 
        # if tot:
    # plot_func(X=xs, Y=ys, Z=Ntot_steadystate[num], cmap='YlGnBu_r')#, vmin=0, vmax=200)#plot the surface of the QLL
    pass


num_steps = len(test_f2d_sim._results['t'])
import matplotlib.animation as animation
ani = animation.FuncAnimation(fig , update_fig, num_steps, interval=50, blit=False, cache_frame_data=False, repeat = True)
plt.show()

In [ ]:
#graph difference in steps over time: should converge to zero if steady state is reached
Fliq_ss, Ntot_steadystate = test_f2d_sim.steady_state_calc()
nice_ss = Ntot_steadystate-Fliq_ss
print(np.shape(Ntot_steadystate))
xs, ys = np.meshgrid(test_f2d_sim.x, test_f2d_sim.y)

fig = plt.figure(figsize=(12,12))
ax = plt.axes(projection = '3d')
#ax.plot_surface(test_f2d_sim._results['t'], Ntot_steadystate[:,])
ax.set_xlabel(r'$x (\mu m$)')#,fontsize=fontsize)
ax.set_ylabel(r'$y (\mu m$)')#,fontsize=fontsize)
ax.set_zlabel(r'$ice \ layers$')#,fontsize=fontsize)
#limits
#ax.set_zlim3d(-self.layermax, self.layermax)
#ax.set_ylim(0, max(self.y))
#ax.set_xlim(0, max(self.x))
def update_fig(num):
    ax.clear() # remove last iteration of plot 
    plot_func = ax.plot_surface
    # if crossSection:
    #     xmid = round(np.shape(Nice)[0]/2)
    #     if ice:
    #         plot_func(X=xs[xmid:], Y=ys[xmid:], Z=nice_ss[num][xmid:][:],cmap ='viridis')# cmap='viridis')#, vmin=0, vmax=200) #plot half of the surface of the ice
    #     if tot:
    #         plot_func(X=xs[xmid:], Y=ys[xmid:], Z=Ntot_steadystate[num][xmid:][:], cmap='cividis')#, vmin=0, vmax=200) #plot half the surface of the QLL
    # else:
    #     if ice:
    plot_func(X=xs, Y=ys, Z=nice_ss[num], cmap='viridis')#, vmin=0, vmax=200) #plot the surface of the ice 
        # if tot:
    # plot_func(X=xs, Y=ys, Z=Ntot_steadystate[num], cmap='YlGnBu_r')#, vmin=0, vmax=200)#plot the surface of the QLL
    pass


num_steps = len(test_f2d_sim._results['t'])
import matplotlib.animation as animation
ani = animation.FuncAnimation(fig , update_fig, num_steps, interval=50, blit=False, cache_frame_data=False, repeat = True)
plt.show()

In [ ]:
test_2d_sim = Simulation(f2d, (500,500), rtol=1e-12)
test_2d_sim.layermax = 2
#test_2d_sim = loadSim('f2d_simulation_august13.pkl')
_plot = test_2d_sim.plot()

1d simulation 

In [ ]:
test_2d_sim.save(id=['august13'])

In [ ]:
fig = plt.figure()
ax = plt.axes(projection='3d')
num_steps = len(test_2d_sim.results()['t'])
xs,ys = np.meshgrid(test_2d_sim.x, test_2d_sim.y)
ax.plot_surface(X=xs, Y=ys, Z=test_2d_sim.getNtot()[num_steps-1], cmap='viridis')

In [ ]:
test_1d = Simulation(f1d, shape=(500,), rtol=1e-12)
test_1d.layermax = 500
test_1d.results()

In [ ]:
#test_1d.animate(ice=True, tot=True)
test_1d.plot()

2d Idea: increase sigmastepmax to see if faceted growth occur, removing the concave "bowl" forming at the center of initially tested 2d systems (test_f2d_sim)

In [ ]:
# sigma_test = Simulation(f2d, (500,500),rtol = 1e-12)
# sigma_test.layermax = 100
# sigma_test.sigmastepmax = 0.25
# sigma_test.results()

In [ ]:
#sigma_id = ['sigmastepmax_'+str(round(100*sigma_test.sigmastepmax)), str(sigma_test.layermax)+'layers']
#sigma_test.save(id=sigma_id)

#test increased layers with increased sigmastep

#sigma_test = loadSim('f2d_simulation_sigmastepmax_025.pkl') # 20 layers?
#sigma_test = loadSim('f2d_simulation_sigmastepmax_025_50layers.pkl')
sigma_test = loadSim('f2d_simulation_sigmastepmax_25_100layers.pkl')

In [ ]:
sigma_test.animate()

2d Ablation testing

In [ ]:
f2d_ablate = Simulation(f2d, (500,500), rtol=1e-12)
f2d_ablate.sigmastepmax = -0.1
f2d_ablate.layermax = 50
f2d_ablate.animate()

1d growth testing. Works, however does not currently match the literature graph despite using the same parameters.

In [ ]:
f1d_sim = Simulation(f1d, (500,), rtol=1e-12)

In [ ]:
#change parameters to match Gladich et al paper's graph and run simulation
f1d_sim.layermax = 500#10000
#f1d_sim.center_reduction = 0.25 #is default now

In [ ]:
f1d_sim.run()

In [ ]:
fig = f1d_sim.plot(completion=0.99, ice=True, tot=True)

In [ ]:
#print variables in 1d simulation for comparison to literature
for k,v in f1d_sim._extra_vars.items():
    print(k,v)
print('center reduction %: ', f1d_sim.center_reduction)

In [ ]:
#functions tested so far
#test_f2d_sim.countermax_0D
#test_f2d_sim.countermax_1D
#test_f2d_sim.countermax_2D
#test_f2d_sim.atol
#test_f2d_sim.rtol
#test_f2d_sim.deltaT
#test_f2d_sim.float_params
#test_f2d_sim.animate()
#test_f2d_sim.run()
#test_f2d_sim.steady_state_calc()
#test_f2d_sim.plot()
#test_f2d_sim.save()
#test_f2d_sim.load() #WORKS to load simulation from file if simulation object exists already -- for loading straght froom file use loadSim()
#loadSim() #works



In [ ]:
num_steps = len(test_f2d_sim.results()['t'])
#shape of results is (num_steps, 2, nx, ny)
Fliq, Ntot = [],[]
for step in range(num_steps):
    next_Fliq, next_Ntot = test_f2d_sim._results['y'][step]
    Fliq.append(next_Fliq)
    Ntot.append(next_Ntot)    
Fliq,Ntot = np.array(Fliq), np.array(Ntot)
Nice = Ntot - Fliq
#shape of fliq, ntot and nice should be (num_steps, nx, ny)

#access coordinate arrays for plotting
xs, ys = np.meshgrid(test_f2d_sim.x, test_f2d_sim.y)

In [ ]:
print(num_steps)
print(test_f2d_sim.steepness(0,slice(0,test_f2d_sim.shape[0]//2)))

In [ ]:
print(test_f2d_sim.get_step_density(0,slice(0,test_f2d_sim.shape[0]//2)))  